# Altair Practice Lab

In this assignment we will be crafting a series of visualizations using
Altair to get practice working with real data in this context.

Your responses should be within the functions given,
using appropriate helper functions to help with clarity
and reduce redundancy.

## Rubric

The criteria to receive an S on this assignment is a good-faith attempt at each portion.

A good-faith attempt should either:

- be fundamentally correct producing the expected output with minimal deviations,
- **OR** contain an explanation of what does not work and _details on what was tried_.

**To receive an N, at least half of the assignment should have good-faith attempts.**

Your charts do not need to match the examples exactly!  They are helpful to get a sense of what you're after, but focus on the problem description.

## Introducing the Dataset

In the data directory you'll find three files:

**legislators.csv** which consists of ~7400 records representing state legislators, it has the following fields:

- name
- given_name
- family_name
- party: As reported by the state.
- gender: Male / Female / Other\*
- jurisdiction: This field contains an identifier for the state or jurisdiction, see below for details.
- district: the name of the district represented
- type: upper | lower - The classification of the legislative. Most states have both, but DC and NE only have an upper chamber.

Note: Accurate data on gender is hard to come by in many states. There may be irregularities in this field. This is also why this field does not make further distinctions beyond Male/Female/Other.

**populations.csv** 

A CSV file with a row with each state's population.  

**actions.csv**

A CSV file with legislative actions for a few states across a range of years:

- description: action text as reported by state
- date: date of action recorded
- classification: not used for this assignment
- state: 2-letter state postal code (note: these are upper case, other files use lower case)
- session: an identifier representing which session the action comes from (not used)

In [60]:
import altair as alt
import pandas as pd
import polars as pl
import re

## Part 1: Data Exploration

First, we'll build a few exploratory visualizations to get a sense of the data for this assignment.

### 1.0: Cleaning

As mentioned above, there is no 'state' field. Jurisdiction is in the format:

`ocd-jurisdiction/country:us/state:nc/government`

and for non-states:

- `ocd-jurisdiction/country:us/district:dc/government`
- `ocd-jurisdiction/country:us/territory:pr/government`

So for our purposes, we want to add a `state` column from the two letter code after either "state:", "district:", or "territory:".
(We will treat DC and PR as states.)

Complete the function `legislators_df` which should return the data from `legislators.csv` in a dataframe, with an additional `state` column.

In [61]:
def legislators_df():
    # Load the data from a CSV file into a DataFrame
    df = pd.read_csv('data/legislators.csv')
    
    # Use regex to extract the state code from the 'jurisdiction' column
    # The regex looks for 'state', 'district', or 'territory', followed by a two-letter code
    # Then it converts the code to uppercase
    df['state'] = df['jurisdiction'].str.extract(r'(?:state|district|territory):([a-z]{2})')[0].str.upper()
    
    return df

# render dataframe
legislators_df()

,name,given_name,family_name,gender,party,jurisdiction,district,type,state
0,Robert White,Robert C.,White,Male,Independent,ocd-jurisdiction/country:us/district:dc/govern...,At-Large,legislature,DC
1,Charles Allen,Charles,Allen,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Ward 6,legislature,DC
2,Vince Gray,Vincent Condol,Gray,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Ward 7,legislature,DC
3,Phil Mendelson,Philip Heath,Mendelson,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Chairman,legislature,DC
4,Trayon White,Trayon,White,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Ward 8,legislature,DC
...,...,...,...,...,...,...,...,...,...
7430,Sol Higgins,Sol Yamiz,Higgins Cuadrado,Female,Partido Popular Democrático,ocd-jurisdiction/country:us/territory:pr/gover...,35,lower,PR
7431,Migdalia González,Migdalia I.,Gonzalez Arroyo,Female,Partido Popular Democrático,ocd-jurisdiction/country:us/territory:pr/gover...,4,upper,PR
7432,Keren Riquelme,Keren L.,Riquelme Cabrera,Female,Partido Nuevo Progresista,ocd-jurisdiction/country:us/territory:pr/gover...,At-Large,upper,PR
7433,Jocelyne Rodríguez Negrón,Jocelyne M.,Rodriguez Negron,Female,Partido Popular Democrático,ocd-jurisdiction/country:us/territory:pr/gover...,19,lower,PR


### 1.1: Initial Plot

First let's build a visualization of gender breakdowns in state legislatures.
Use the following:

- stacked bars per state
- each segment of stacked bar is gender

Your graph should somewhat resemble *imgs/ex1.1.png*.

In [62]:
# Increase row limit in Altair
alt.data_transformers.enable("csv")

#https://altair-viz.github.io/user_guide/large_datasets.html

DataTransformerRegistry.enable('csv')

In [63]:
def states_by_gender_initial(df):
    chart = alt.Chart(df).mark_bar().encode(
        x=alt.X('state:N', axis=alt.Axis(labelAngle=-90)),
        y=alt.Y('count():Q'),
        color=alt.Color('gender:N'),
        order=alt.Order('gender:N', sort='ascending')
    )
    return chart

# render chart
states_by_gender_initial(legislators_df())

alt.Chart(...)

References: 

https://altair-viz.github.io/altair-viz-v4/user_guide/encoding.html#encoding-channel-options

https://altair-viz.github.io/gallery/stacked_bar_chart.html 

https://stackoverflow.com/questions/52877697/order-bar-chart-in-altair 

https://altair-viz.github.io/user_guide/generated/channels/altair.Y.html


### 1.2: Improvements

While it is clear from the first chart that there are more elected officials that are men than women, it is hard to compare across states.

Make the following adjustments:

- Normalize the chart so that each bar is a percentage, allowing for direct comparison across states.
- Since this is US political data, the colors red and blue have a strong meaning, associated with the Republican and Democratic parties. Change the color scheme to avoid red and blue. (I chose #8624f5 for women and #1fc3aa for men based on this article: <https://blog.datawrapper.de/gendercolor/>)
- Two states are very close to 50%, add a line at 50% using a layered chart to make it easier to see if they exceed 50% or not.

Your graph should somewhat resemble *imgs/ex1.2.png*.

In [64]:
def states_by_gender_improved(df):
    # Create the bar chart
    chart = alt.Chart(df).mark_bar().encode(
        x=alt.X('state:N', axis=alt.Axis(labelAngle=-90)),
        y=alt.Y('count()', stack="normalize"),
        color=alt.Color('gender:N', 
                        scale=alt.Scale(domain=['Female', 'Male'], range=['#8624f5', '#1fc3aa'])
                       ),
        order=alt.Order('gender:N', sort='ascending')  
    )

    # Create the horizontal rule at 50%
    rule = alt.Chart(pd.DataFrame({'y': [0.5]})).mark_rule(color='gray').encode(y='y')
    
    final_chart = chart + rule
    
    return final_chart


states_by_gender_improved(legislators_df())

alt.LayerChart(...)

Reference: 

https://altair-viz.github.io/altair-viz-v4/gallery/normalized_stacked_bar_chart.html

https://github.com/vega/altair/issues/921

https://altair-viz.github.io/gallery/layer_line_color_rule.html

https://stackoverflow.com/questions/71034458/how-to-make-the-mark-rule-in-altair-change-based-on-user-input



## Part 2: Party Breakdown

We'll now take a look at party control. We can start with essentially the same chart.

### 2.0 - Party Control

Copy your code from 1.2 above & modify it to use party instead of gender.  Your graph will wind up with too many parties, see `imgs/ex2.0.png`.

In [65]:
def party_control_raw(df):
    # Create the bar chart
    chart = alt.Chart(df).mark_bar().encode(
        x=alt.X('state:N', axis=alt.Axis(labelAngle=90)),
        y=alt.Y('count()', stack="normalize"),
        color=alt.Color('party:N'),
        order=alt.Order('party:N', sort='ascending')  
    )

    # Create the horizontal rule at 50%
    rule = alt.Chart(pd.DataFrame({'y': [0.5]})).mark_rule(color='gray').encode(y='y')
    
    
    final_chart = chart + rule
    
    return final_chart

party_control_raw(legislators_df())

alt.LayerChart(...)

### 2.1 - Cleaning Data

The above graph still has some shortcomings:

- Most states have an upper and lower chamber, and party control may vary between them. We'll need to make two bars per state (which we'll tackle in 2.2).
- Also, there are too many variations of party as you can see here:

Let's transform the data again, adding a new column "party_code" with the following rules:

- if the word 'Democratic' appears, set party_code to 'D'
- if the word 'Republican' appears, set the party_code to 'R'
- otherwise, set the party_code to 'O'

Party data in NE, DC, and PR does not work with this scheme.
For simplicity, we will exclude them from our analysis.

For this portion, implement `clean_party_df` which should return a modified legislators DataFrame with the `party_code` column, and the rows for states 'DC', 'NE' and 'PR' dropped.

In [66]:
def clean_party_df():
    # start with the DataFrame from part 1 & return transformed copy
    df = legislators_df()
    df = df[~df['state'].isin(['DC', 'NE', 'PR'])]
    
    # Create a new column 'party_code' by applying a lambda function to the 'party' column
    #check for Democratic and change to D, check for Republican and change to R, elso 0
    df['party_code'] = df['party'].apply(lambda x: 'D' if 'Democratic' in x else ('R' if 'Republican' in x else 'O'))

    return df

clean_party_df()

,name,given_name,family_name,gender,party,jurisdiction,district,type,state,party_code
13,Kevin McCabe,Kevin J.,McCabe,Male,Republican,ocd-jurisdiction/country:us/state:ak/government,30,lower,AK,R
14,C.J. McCormick,Conrad J.,McCormick,Male,Democratic,ocd-jurisdiction/country:us/state:ak/government,38,lower,AK,D
15,Bill Wielechowski,Bill P.,Wielechowski,Male,Democratic,ocd-jurisdiction/country:us/state:ak/government,K,upper,AK,D
16,Justin Ruffridge,Justin,Ruffridge,Male,Republican,ocd-jurisdiction/country:us/state:ak/government,7,lower,AK,R
17,Will Stapp,Will,Stapp,Male,Republican,ocd-jurisdiction/country:us/state:ak/government,32,lower,AK,R
...,...,...,...,...,...,...,...,...,...,...
7353,Karlee Provenza,Karlee R.,Provenza,Female,Democratic,ocd-jurisdiction/country:us/state:wy/government,45,lower,WY,D
7354,Affie Ellis,Affie Burnside,Ellis,Female,Republican,ocd-jurisdiction/country:us/state:wy/government,8,upper,WY,R
7355,Abby Angelos,Abby,Angelos,Female,Republican,ocd-jurisdiction/country:us/state:wy/government,3,lower,WY,R
7356,Pepper Ottman,Pepper L.,Ottman,Female,Republican,ocd-jurisdiction/country:us/state:wy/government,34,lower,WY,R


### 2.2 - Faceted Plot

Add a function `party_control_by_chamber` that contains the following elements:

- One bar per state, **along the Y axis**.
- Each bar should consist of a stack: a blue portion, a green portion, and a red portion, corresponding to the D, O, and R `party_code` respectively.
- A vertical line at the 50% mark, indicating (likely) party control.
- Finally, facet the chart on `type` so that you get a set of bars for the lower and upper chambers.

See `imgs/ex2.2.png` for an example.

In [67]:
def party_control_by_chamber(df):
    # Bar chart for normalized counts
    charts = alt.Chart().mark_bar().encode(
        x=alt.X('count():Q', stack="normalize"),
        y= alt.Y('state:N'),
        color=alt.Color('party_code:N', 
                        scale=alt.Scale(domain=['R', 'D', 'O'], 
                                        range=['darkred', 'lightblue', 'lightgreen']))
    )

    # Vertical line at x=0.5
    vertline = alt.Chart(pd.DataFrame({'x': [0.5]})).mark_rule(color='gray').encode(
        x='x'
    )

    # Combine the bar chart and the vertical line
    final_chart = alt.layer(
        charts, vertline,
        data=df
    ).facet(
        column='type:N'  # Facet the chart by the 'type' column
    )

    return final_chart

party_control_by_chamber(clean_party_df())


alt.FacetChart(...)

https://github.com/vega/altair/issues/1124
https://altair-viz.github.io/user_guide/compound_charts.html
https://altair-viz.github.io/gallery/grouped_bar_chart.html


## Part 3: Comparing by Population

For part three, we are interested in the relationship of various properties of legislatures to the total population of the state.

To do this, we'll need to create a combined DataFrame that mixes in data from `populations.csv`.

### 3.0 - Create Combined DataFrame

Write the function `population_combined_df`, which should return a DataFrame with the columns:

- state: abbreviation of state
- upper: total seats in upper chamber
- lower: total seats in lower chamber
- pop_2020: the 2020 population, obtained from merging with `population.csv`

**Data Note:** These numbers are based on the non-vacant seats as-of a particular day in September 2024. Vacancies will cause the counts to be off by a bit, but the general size of the legislature should be roughly the same.

In [68]:
def population_combined_df():
    # start with the DataFrame from part 1 & return transformed copy
    df_legislator = legislators_df()

    # Create a pivot table to count legislators in 'upper' and 'lower' chambers by state
    df_seat_counts = df_legislator.pivot_table(
        index='state',
        columns='type',
        values='name',  # Using any non-empty column to count occurrences
        aggfunc='size',  # Counting occurrences
        fill_value=0  # Filling missing values with 0
    )

    # Convert the index 'state' back into a regular column
    df_seat_counts = df_seat_counts.reset_index()
    df_seat_counts = df_seat_counts[['state', 'upper', 'lower']]

    # Load population data
    df_population = pd.read_csv('data/populations.csv')

    # Ensure the 'state' column is in uppercase
    df_population['state'] = df_population['state'].str.upper()

    # Merge seat counts with population data based on 'state'
    df_combined = pd.merge(df_seat_counts, df_population, on='state', how='inner')
    
    return df_combined

population_combined_df()

,state,upper,lower,pop_2020
0,AK,20,40,733374
1,AL,35,104,5024294
2,AR,35,98,3011490
3,AZ,30,59,7157902
4,CA,40,79,39538212
5,CO,35,65,5773707
6,CT,36,151,3605912
7,DC,0,0,689548
8,DE,21,41,989946
9,FL,40,119,21538216


Reference: https://www.codetodevs.com/como-agrupar-resumir-datos-en-pandas/

### 3.1 - Create Population vs. Seats Scatterplot

Create a new plot with two layers:

- Population on the X axis
- Number of seats on the Y axis
- Upper chamber points should be purple and use the 'triangle-up' shape.
- Lower chamber points should be orange and use the 'triangle-down' shape.
- Make a customization or two to your chart's default labels and axes, whatever you feel is appropriate.

Hint: You can layer two charts for this.

In [69]:
def scatter_pop_size():
    df = population_combined_df()  

    # Create the lower chart (pop_2020 vs lower)
    lower_chart = alt.Chart(df).mark_point().encode(
        x='pop_2020:Q',  
        y='lower:Q', 
        shape=alt.value('triangle-down'),  
        color=alt.value('orange')  
    )
    
    # Create the upper chart (pop_2020 vs upper)
    upper_chart = alt.Chart(df).mark_point().encode(
        x='pop_2020:Q',  
        y='upper:Q',  
        shape=alt.value('triangle-up'),  
        color=alt.value('purple')  
    )
    
    # Return the combined charts
    return lower_chart + upper_chart 

scatter_pop_size()
    

alt.LayerChart(...)

Reference: 
https://altair-viz.github.io/user_guide/compound_charts.html#layer-chart
https://altair-viz.github.io/user_guide/marks/point.html

### 3.2 - Regressions

Add two more layers, a purple & orange regression line for each chamber.  See `imgs/ex3.2.png`

Hint: See `transform_regression`.

In [70]:
def scatter_pop_size_regression():
    df = population_combined_df()  

    # Create the lower chart (pop_2020 vs lower) add regression line
    lower_chart = alt.Chart(df).mark_point().encode(
        x='pop_2020:Q',  
        y='lower:Q', 
        shape=alt.value('triangle-down'),  
        color=alt.value('orange')  
    )
    lower_regression = lower_chart.transform_regression('pop_2020', 'lower').mark_line(color='orange') #add regression line
    # Create the upper chart (pop_2020 vs upper) add regression line
    upper_chart = alt.Chart(df).mark_point().encode(
        x='pop_2020:Q',  
        y='upper:Q',  
        shape=alt.value('triangle-up'),  
        color=alt.value('purple')  
    )
    upper_regression = upper_chart.transform_regression('pop_2020', 'upper').mark_line(color='purple') #add regression line

    # Combine the scatter plots and regression lines
    return (lower_chart + lower_regression) + (upper_chart + upper_regression)


scatter_pop_size_regression()


alt.LayerChart(...)

reference: https://altair-viz.github.io/user_guide/transform/regression.html

## Part 4: Actions Heatmap

The file `actions_il-in-mi-wi_2021-2024.csv` contains nearly half a million records, representing every official action taken on a piece of legislation in these four midwestern states over the past two sessions.

Legislatures work quite differently, some meet all year, while others meet for very short periods.
By creating a heatmap of what days actions take place, we can get a sense of how different states compare.

### 4.0 - Load Actions

Complete `actions_df`, which should load the data from `actions_il-in-mi-wi_2021-2024.csv`.

Tips: 
- Make sure that the `date` column is loaded as a date type!
- Dates are in YYYY-MM-DD format, though some also have additional characters for time, which you will want to ignore.

In [71]:
def actions_df():
    df = pd.read_csv('data/actions_il-in-mi-wi_2021-2024.csv')

    # Use regex to extract only the YYYY-MM-DD part and remove the rest
    df['date'] = df['date'].apply(lambda x: re.sub(r'(\d{4}-\d{2}-\d{2}).*', r'\1', x))

    # Convert the cleaned 'date' column to datetime format
    df['date'] = pd.to_datetime(df['date'])

    return df

actions_df()

,description,date,classification,state,session
0,"Filed with the Clerk by Rep. Marcus C. Evans, Jr.",2021-02-10,['filing'],IL,102nd
1,Filed with Secretary,2021-08-26,['filing'],IL,102nd
2,Filed with Secretary by Sen. Jacqueline Y. Col...,2021-02-25,['filing'],IL,102nd
3,Filed with Secretary,2022-01-19,['filing'],IL,102nd
4,Filed with Secretary,2022-11-29,['filing'],IL,102nd
...,...,...,...,...,...
456400,Partial veto failed to pass notwithstanding th...,2022-05-17,[],WI,2021
456401,Fiscal estimate received,2021-02-08,['receipt'],WI,2021
456402,Fiscal estimate received,2021-02-10,['receipt'],WI,2021
456403,Placed on calendar 5-17-2022 pursuant to Joint...,2022-05-17,[],WI,2021


### 4.1 - Actions Heatmap

Generate a heatmap (using `mark_rect`) with:

- a row per state
- each row consists of shaded marks with shading based on the total action count for a given week

Tip: Use the 'yearweek(date)' aggregation for the X channel.

See `imgs/ex4.1.png`.

In [72]:
import altair as alt

def actions_heatmap_scaled(df):
    
    heatmap = alt.Chart(df).mark_rect().encode(
        alt.X("yearweek(date):O", axis=alt.Axis(labelAngle=0, format="%Y-W%U", tickCount=13), title="Date (year-week)"),
        alt.Y("state:N", title="State"),
        alt.Color("count():Q", title='actions')
    ).properties(
        width=1200,
        height=150
    )

    return heatmap


actions_heatmap_scaled(actions_df())


alt.Chart(...)

reference: https://altair-viz.github.io/user_guide/marks/rect.html

### 4.2 - Excluding IL Outliers

Illinois clearly dominates the above graph, below modify two calls to `actions_heatmap` with a modified dataframe that excludes IL, and a modified dataframe that only includes IL.

(Note how by using functions in our dataframe we can more easily reuse portions by making small adjustments to the data.)

See `ex4.2a.png` and `ex4.2b.png`

In [73]:
# MODIFY THIS LINE (exclude IL)
actions_heatmap_scaled(actions_df()[actions_df()['state'] != 'IL'])

alt.Chart(...)

In [74]:
# MODIFY THIS LINE (only IL)
actions_heatmap_scaled(actions_df()[actions_df()['state'] == 'IL'])

alt.Chart(...)

#### 4.3 - Cumulative Line Chart

Another way to view this data would be with a cumulative line chart.

Create a chart with:

- days on the X axis
- cumulative actions to date on the Y axis
- one line per state

Hint: To do this you will need to look at the `transform_window` function.

See `ex4.3.png` for an example.

In [75]:
def actions_cumulative(df):
    chart = alt.Chart(df).transform_window(
        frame=[None, 0],
        sort=[{'field': 'date'}],  # Sort by date to calculate cumulative count correctly
        groupby=['state'],  # Group by state to get cumulative count per state
        cumulative_count='count()'  # Calculate cumulative count of actions
    ).mark_line().encode(
        x=alt.X('yearmonthdate(date):O', axis=alt.Axis(labelAngle=0, format="%b %d, %Y", tickCount=13), title="date(year-month-date)"),
        y=alt.Y('cumulative_count:Q', title="cumulative_actions"),
        color='state:N' 
    ).properties(
        width=800,
        height=300
    )

    return chart

actions_cumulative(actions_df())



alt.Chart(...)

reference: 
https://altair-viz.github.io/user_guide/generated/core/altair.Axis.html

https://altair-viz.github.io/user_guide/transform/window.html